# Project 1

Mama mia, here we go again with classification...

## Imports 

In [4]:
import sys

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns

print(f"Python version: {sys.version}.")

Python version: 3.10.4 (main, Mar 31 2022, 03:38:35) [Clang 12.0.0 ].


## Loading Data

In [12]:
df_train = pd.read_csv("./Data_train.csv")

In [13]:
df_train.head()

,Unnamed: 0,x,y,class
0,0,9.478938,-4.950071,1
1,1,-8.476344,3.635107,3
2,2,4.080771,-3.684994,3
3,3,-0.509729,-5.700655,2
4,4,-0.898476,-4.007790,3


In [14]:
df_train = df_train.drop("Unnamed: 0", axis='columns')
df_train.head()

,x,y,class
0,9.478938,-4.950071,1
1,-8.476344,3.635107,3
2,4.080771,-3.684994,3
3,-0.509729,-5.700655,2
4,-0.898476,-4.007790,3


In [16]:
X_train, y_train = df_train[["x", "y"]], df_train["class"].values.reshape(-1, 1)

Let's do the same for the test data:

In [17]:
df_test = pd.read_csv("./Data_test.csv").drop("Unnamed: 0", axis='columns')
df_test.head()

,x,y,class
0,2.597513,-1.720239,1
1,7.951270,6.252087,2
2,-6.913355,4.858647,3
3,-2.948122,-0.585784,3
4,0.865727,-3.874243,3


In [18]:
X_test, y_test = df_test[["x", "y"]], df_test["class"].values.reshape(-1, 1)

## Task 0: Naive Logistic Regression